In [1]:
# Model Arguments 
from config import ModelArguments

config = ModelArguments()

# Multimodal Encoder, Resampler, Projector
from multimodal_encoder.builder import build_vision_tower
from multimodal_resampler.builder import build_vision_resampler
from multimodal_projector.builder import build_vision_projector

vision_tower = build_vision_tower(config)
vision_resampler = build_vision_resampler(config)
vision_projector = build_vision_projector(config)

# Llava Model :: inherit from transformers.PreTrainedModel class unique initialization with config
# from llava_llama import LlavaConfig, LlavaLlamaModel

# llava_config = LlavaConfig()
# llava_llama_model = LlavaLlamaModel(llava_config) # took around 2 min to initialize already (time to move to RunPod?)

On MacOS, decord package is not compatible with new python version.
Loading vision tower: openai/clip-vit-base-patch32


/opt/homebrew/anaconda3/lib/python3.11/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [2]:
# from llava_llama import LlavaLlamaForCausalLM

# LlavaLlamaForCausalLM(llava_config)

In [ ]:
from language_model.llava_llama import LlavaLlamaForCausalLM # Register the llava models into 'transformers'
# from llava_arch import LlavaMetaModel
from transformers import AutoConfig
from transformers import AutoModelForCausalLM

config = AutoConfig.for_model("llava_llama", trust_remote_code=True)
model = AutoModelForCausalLM.from_config(config, trust_remote_code=True)

In [3]:
import av
container = av.open("vid.mp4")
for frame in container.decode(video=0):
    img_pil = frame.to_image()
    break

In [2]:
from dataprocess import LazySupervisedDataset
from transformers import AutoTokenizer
from config import DataArguments 

data_args = DataArguments(
    data_path = "mock.json",
    image_folder = "",
    video_folder = "",
    video_fps = 1,
    frames_upbound = 0,
    add_time_instruction = False,
    force_sample = False,
    default_fps = 10
)

tokenizer = AutoTokenizer.from_pretrained("meta-llama/Meta-Llama-3-8B-Instruct")
dataset = LazySupervisedDataset(data_args=data_args, tokenizer=tokenizer, image_processor=vision_tower.image_processor)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [3]:
source = dataset.data[0]
# 'image' in source

In [4]:
dataset[0]

{'input_ids': tensor([128000, 128006,   9125, 128007,    271,   2675,    527,    264,  11190,
           4221,    323,  11376,  18328,     13,   1472,    527,   3025,    311,
           3619,    279,   9302,   2262,    430,    279,   1217,   5825,     11,
            323,   7945,    279,   1217,    449,    264,   8205,    315,   9256,
           1701,   5933,   4221,     13, 128009, 128006,    882, 128007,    271,
           -200,    198,   3923,    649,    499,   1518,    304,    420,   2217,
             30, 128009, 128000, 128006,  78191, 128007,    271, 128000, 128009,
         128006,    882, 128007,    271,  11787,   1070,    904,   1274,    304,
            279,   2217,     30, 128009, 128000, 128006,  78191, 128007,    271,
         128000, 128009]),
 'labels': tensor([128000, 128006,   -100, 128007,   -100,   -100,   -100,   -100,   -100,
           -100,   -100,   -100,   -100,   -100,   -100,   -100,   -100,   -100,
           -100,   -100,   -100,   -100,   -100,   -100,   